# Working with symbolic expressions

## Contents
+ Modeling algebraic expressions as data structures
+ Writing code to analyze, transform, or evaluate algebraic expressions
+ Finding the derivative of a function by manipulating the expression that defines it
+ Writing a Python function for automatic derivatives
+ Using *SymPy* library for automatic integrals

## Introduction

In the previous chapters we've used Python to approximate solutions for two of the most important concepts in calculus: derivatives and integrals. 

We learned to calculate derivatives by taking slopes of smaller and smaller secant lines, and to calculate integrals by estimating the area under a graph with thinner and thiner rectangles.

By using that *programmatic* approach, we've been able to work do very interesting applications without following the *formal* approach first.

That *formal* (non-programmatic) approach dictates how to calculate the exact formula for a derivative and integral.
For example, if $ f(x) = x^3 $, $ f'(x) = 3x² $.

### Finding an exact derivative with a computer algebra system

While knowing the rules for derivatives and integrals are very important for Math purposes, those are actually not that useful for computing.

In case you need to know the exact formula for a derivative, you can always rely on a *computer algebra system* like *Mathematica*. In particular, there are tools online such as [wolframalpha.com](https://www.wolframalpha.com/), which are powered by *Mathematica* engine.

For instance:

![Wolfram Alpha](../images/wolfram_alpha.png)

This will obviously will work for more complicated formulas such as:

$
\displaystyle
f(x) = \frac{1}{1 + e^{-x}}
$

Which would be written in Python as: `1/(1 + e**(-x))` can be punched into [wolframalpha.com](https://www.wolframalpha.com/) to obtain:

$
\displaystyle
f(x) = \frac{e^{-x}}{(1 + e^{-x})^2}
$

In short, from a programmer's POV, it is more important to know that the derivative represents the instantaneous rate of change of a given function, and know that you have tools such as [wolframalpha.com](https://www.wolframalpha.com/) to obtain the derivative that remembering all the derivative rules by heart.

Note that *Mathematica* does not use the approximation approach we've been learning in the past chapters. Instead, it leverages a technique called *symbolic programming* to come up with the formulas.

In this chapter, we will learn how to do some *symbolic programming* in Python to manipulate algebraic formulas directly, and ultimately figure out the formulas for the derivatives.

### Doing symbolic algebra in Python

When representing a mathematical formula in Python using the common representation we've used until now, which consists in defining a Python function, does not let us obtain a lot of introspection info about the formula itself:

```python
from math import sin

def f(x):
    return (3 * x ** 2 + x) * sin(x)
```

We can use the representation above to calculate data points, and plot the function, but that representation will make it very difficult for us to create a function such as `contain_division(f)` that would return `False` in the case above.

In order to solve such questions we need to represent the function as a *data structure* that would help us understand the algebraic expression that defines the function at a deeper level than the Python function does, while also give us a chance to manipulate it (for example to expand the product).

> Our goal for the chapter is to build a program that takes an algebraic expression and transforms it into a data structure that can represent variables, numbers, sums, differences, products, quotients, powers and special functions such as the sine and cosine. With that, we will also allow the generation of derivatives for all of them.

## Modeling algebraic expressions

Let's consider the function $ f(x) = (3x² + x) \cdot sin(x) $. This is a good candidate for finding the correct modeling approach because it contains variables, numbers, addition and multiplication, a power and special functions such as *sine*.

The first note is that the function name $ f $ is an arbitrary name and does not affect the definition of the function. Thus, we can focus on the right-hand side of the equation.

> The right hand side of an equation is called an *expression* &mdash; a collection of mathematical symbols combine in some valid way.

### Breaking an expression into pieces

Now that it's clear that we will be focusing in $ (3x² + x) \cdot sin(x) $, we need to break this expression into smaller pieces.

To properly break that expression, we need to split it by the $ \cdot $ operator:

![Breaking up an algebraic expression](../images/breaking_up_expression.png)



Likewise, $ 3x² + x $ can be broken down into the sum of $ 3x² $ and $ x $. And $ 3x^2 $ can be further decomposed into the product of $ 3 $ and $ x^2 $.

In the terminology of functional programming, functions combining smaller objects into bigger ones are often called *combinators*. The ones that we've found in our *alebraic expression* are:
+ $ 3x² $ is the product of the expressions $ 3 $ and $ x^2 $
+ $ x^2 $ is a power: the expression $ x $ raised to the power of another expression $ 2 $.
+ The expression $ sin(x) $ is a function application &mdash; given the expression $ sin $ and the expression $ x $, we build a new expression $ sin(x) $.

A variable like $ x $, a number like $ 2 $ or a function name like $ sin $ can't be broken down further. To distinguish those from *combinators* we call them *elements*.

### Building an expression tree

The elements and combinators we find in the *algebraic expression* $ (3x² + x) \cdot sin(x) $, along with the concept of applying a function, are sufficient to rebuild the expression as a tree. Let's see how, first manually.

We can start with $ x^2 $, which combines the elements $ x $ and $ 2 $ with the power combinator.

That can be followed by the product of $ 3 $ and $ x^2 $, and so forth until we end up with a whole tree that represents the *algebraic expression* as seen below:

![Building the expression tree](../images/expression_tree_sequence.png)

The root of the tree is the product combinator we used in the previous subsection. From that root node, we create two branches and recursively start creating additional branches that represent the expression.

### Translating the expression tree to Python

In this section we will build some classes representing the expression as a data structure. We will start simple, and start adding more and more capabilities along the way until we reach our goal.


In our implementation, we will model combinators as containers that hold their inputs.

For example:

In [ ]:
class Power():
    def __init__(self, base, exponent):
        self.base = base
        self.exponent = exponent

We also need classes to represent numbers and variables, as those will give us extra flexibility in the implementation:

In [ ]:
class Number():
    def __init__(self, number):
        self.number = number

class Variable():
    def __init__(self, symbol):
        self.symbol = symbol

In [ ]:
With these three classes defined, we will be able to define our first algebraic expression:

In [1]:
class Power():
    def __init__(self, base, exponent):
        self.base = base
        self.exponent = exponent

class Number():
    def __init__(self, number):
        self.number = number

class Variable():
    def __init__(self, symbol):
        self.symbol = symbol

Power(Variable('x'), Number(2))        

Now let's address the `Product` class:

In [ ]:
class Product():
    def __init__(self, expr1, expr2):
        self.expr1 = expr1
        self.expr2 = expr2

With that, we can now model expressions such as $ 3 \cdot x^2 $:

In [2]:
class Power():
    def __init__(self, base, exponent):
        self.base = base
        self.exponent = exponent

class Number():
    def __init__(self, number):
        self.number = number

class Variable():
    def __init__(self, symbol):
        self.symbol = symbol

class Product():
    def __init__(self, expr1, expr2):
        self.expr1 = expr1
        self.expr2 = expr2

Product(Number(3), Power(Variable('x'), Number(2)))

After that, we're in a good position to have our first implementation of our algebraic expression classes that would represent the expression:

$$
\displaystyle
(3x² + x) \cdot sin(x)
$$

In [5]:
class Power():
    def __init__(self, base, exponent):
        self.base = base
        self.exponent = exponent

class Number():
    def __init__(self, number):
        self.number = number

class Variable():
    def __init__(self, symbol):
        self.symbol = symbol

class Product():
    def __init__(self, expr1, expr2):
        self.expr1 = expr1
        self.expr2 = expr2

class Sum():
    def __init__(self, *exprs):
        self.exprs = exprs

class Function():
    def __init__(self, name):
        self.name = name

class Apply():
    def __init__(self, function, argument):
        self.function = function
        self.argument = argument

#  (3x² + x) · sin(x)
expression = Product(
    Sum(
        Product(
             Number(3),
             Power(
                 Variable('x'), 
                 Number(2))), 
        Variable('x')),
    Apply(
        Function('sin'), 
        Variable('x')))        

With those simple classes we have given the first step for the creation of our *algebraic expression engine*. Note that despite having been created for a particular expression, the approach is quite flexible as other expressions such as:

```python
Apply(Function("cos"),Sum(Power(Variable("x"),Number("3")), Number(-5)))
```

faithfully represent other *algebraic expressions* such as:

$$
\displaystyle
cos(x^3 - 5)
$$